In [ ]:
import numpy as np
import os
import copy
import matplotlib.pyplot as plt

In [ ]:
# extract zip files

from zipfile import ZipFile

files = 'docs.zip'

with ZipFile(files,'r') as zip:
    zip.extractall()

# Locality Sensitive Hashing

In [ ]:
def lsh(klasor_adi, ext_arr, k, b, r, similarity_threshold):
    file_count = 0
    for path in os.listdir(klasor_adi):
        if os.path.isfile(os.path.join(klasor_adi, path)):
            file_count += 1
    
    shingles = [] # stored as words
    for f in range(0,file_count):
        file = open(klasor_adi + "/" + str(1000+f) + ".txt", "r").read()
        
        if ',' in file:
            file = file.replace(',', '')
        if '.' in file:
            file = file.replace('.', '')
        if ';' in file:
            file = file.replace(';', '')
        if ':' in file:
            file = file.replace(':', '')
        if '!' in file:
            file = file.replace('!', '')
        if '?' in file:
            file = file.replace('?', '')
        if '\n\n' in file:
            file = file.replace('\n\n', ' ')
        if '\n' in file:
            file = file.replace('\n', ' ')
        
        # Shingling
        file = file.lower()
        arr = file.split(" ")
        k_words = []
        for i in range(len(arr)-k):   
            s = ""
            for j in range(k):
                s += " " + arr[i+j]
            k_words.append(s[1:len(s)])
        shingles.append(k_words)
    
    if len(ext_arr) != 0: # for sorgu.txt
        shingles.append(ext_arr)
    
    # Creating the input matrix (shingles x documents)
    unique_shingles = []
    for shingle_list in shingles:
        for shingle in shingle_list:
            if shingle not in unique_shingles:
                unique_shingles.append(shingle)
    
    input_matrix = []
    for shingle_list in shingles:
        shingles_01 = {shingle: 1 if shingle in shingle_list else 0 for shingle in unique_shingles}
        input_matrix.append(list(shingles_01.values()))
    
    # Min Hashing
    hashes = []
    n_shingles = np.arange(1,len(input_matrix[0])+1)
    np.random.shuffle(n_shingles)
    arr = n_shingles
    for i in range(b*r):
        hashes.append(copy.deepcopy(arr))
        np.random.shuffle(arr)
    
    signature_matrix = []
    for min_hash in hashes:#b x r
        one_signature = []
        for doc in range(len(input_matrix)):
            for h in range(1,len(min_hash)):
                if input_matrix[doc][(np.where(min_hash == h))[0][0]] > 0:
                    one_signature.append(h)
                    break
        signature_matrix.append(one_signature)
    
    # LSH
    buckets = {}
    doc_id = 0
    for column in range(len(signature_matrix[0])):
        band_del = 0 # band delimeter to seperate bands
        for band in range(b):
            band_values = tuple(signature_matrix[row][column] for row in range(band_del, band_del + r))
            band_del += r

            bucket_id = "".join(map(str, band_values))

            if bucket_id in buckets:
                buckets[bucket_id].append(doc_id)
            else:
                buckets[bucket_id] = [doc_id]
        doc_id += 1
    
    return buckets, signature_matrix, similarity_threshold, doc_id

## LSH Query to Find Similar Documents

In [ ]:
def query(input_file):
    k = 3
    file = open(input_file, "r").read()
        
    if ',' in file:
        file = file.replace(',', '')
    if '.' in file:
        file = file.replace('.', '')
    if ';' in file:
        file = file.replace(';', '')
    if ':' in file:
        file = file.replace(':', '')
    if '!' in file:
        file = file.replace('!', '')
    if '?' in file:
        file = file.replace('?', '')
    if '\n\n' in file:
        file = file.replace('\n\n', ' ')
    if '\n' in file:
        file = file.replace('\n', ' ')

    # Shingling
    file = file.lower()
    arr = file.split(" ")
    k_words = []
    for i in range(len(arr)-k):   
        s = ""
        for j in range(k):
            s += " " + arr[i+j]
        k_words.append(s[1:len(s)]) 
    
    buckets, signature_matrix, sim_thres, this_doc = lsh("docs",k_words,k,5,2,0.4)
    sim_buckets = []
    sim_docs = []
    for bucket_id, documents in buckets.items():
        for doc in documents:
            if doc == this_doc:
                sim_buckets.append(documents)
                
    for i in range(len(sim_buckets)):
        for j in range(len(sim_buckets[i])):
            if (sim_buckets[i][j] != this_doc) & (sim_buckets[i][j] not in sim_docs):
                sim_docs.append(sim_buckets[i][j])

    similarity = {}
    input_sig = tuple(signature_matrix[row][this_doc-1] for row in range(len(signature_matrix)))
    for similar in sim_docs:
        signature = tuple(signature_matrix[row][similar] for row in range(len(signature_matrix)))
        
        # Jaccard Similarity
        union = 0
        inters = 0
        for value in range(len(input_sig)):
            if (input_sig[value] != 0) | (signature[value] != 0):
                union += 1
                if input_sig[value] == signature[value]:
                    inters += 1
        similarity[similar] = [float(float(inters)/float(union))]
    
    sorted_sim = {k: v for k, v in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    if not sorted_sim:
        print("Benzer dosya bulunamamıştır.")
    else:
        count = 0
        for doc_id, jaccard in sorted_sim.items():
            count += 1
            print(str(1000+int(doc_id)) + ".txt " + str(jaccard[0]))
        print("Toplam " + str(count) + " dosya bulunmuştur.")
                
query("sorgu.txt")

In [ ]:
def find_sim(input_file, k, b, r):
    file = open(input_file, "r").read()
        
    if ',' in file:
        file = file.replace(',', '')
    if '.' in file:
        file = file.replace('.', '')
    if ';' in file:
        file = file.replace(';', '')
    if ':' in file:
        file = file.replace(':', '')
    if '!' in file:
        file = file.replace('!', '')
    if '?' in file:
        file = file.replace('?', '')
    if '\n\n' in file:
        file = file.replace('\n\n', ' ')
    if '\n' in file:
        file = file.replace('\n', ' ')

    # Shingling
    file = file.lower()
    arr = file.split(" ")
    k_words = []
    for i in range(len(arr)-k):   
        s = ""
        for j in range(k):
            s += " " + arr[i+j]
        k_words.append(s[1:len(s)])
    
    buckets, signature_matrix, sim_thres, this_doc = lsh("docs",k_words,k,b,r,0.4)
    sim_buckets = []
    sim_docs = []
    for bucket_id, documents in buckets.items():
        for doc in documents:
            if doc == this_doc:
                sim_buckets.append(documents)
               
    for i in range(len(sim_buckets)):
        for j in range(len(sim_buckets[i])):
            if (sim_buckets[i][j] != this_doc) & (sim_buckets[i][j] not in sim_docs):
                sim_docs.append(sim_buckets[i][j])
    
    similarity = {}
    input_sig = tuple(signature_matrix[row][this_doc-1] for row in range(len(signature_matrix)))
    for similar in sim_docs:
        signature = tuple(signature_matrix[row][similar] for row in range(len(signature_matrix)))
        
        # Jaccard Similarity
        union = 0
        inters = 0
        for value in range(len(input_sig)):
            if (input_sig[value] != 0) | (signature[value] != 0):
                union += 1
                if input_sig[value] == signature[value]:
                    inters += 1
        similarity[similar] = [(inters/union)]
    return similarity
    
def lsh_count(input_file):
    # 1
    x = ["k=2", "k=3", "k=4"]
    y = []
    for i in range(2,5):
        lsh_ = find_sim(input_file,i,5,4)
        y.append(len(lsh_))
    
    plt.bar(x, y)
    plt.xlabel('k values')
    plt.ylabel('number of similar documents')
    plt.show()
    
    # 2
    x = ["b=3", "b=4", "b=5"]
    y = []
    for i in range(3,6):
        lsh_ = find_sim(input_file,3,i,4)
        y.append(len(lsh_))
    
    plt.bar(x, y)
    plt.xlabel('band values')
    plt.ylabel('number of similar documents')
    plt.show()
    
    # 3
    x = ["r=3", "r=4", "r=5"]
    y = []
    for i in range(3,6):
        lsh_ = find_sim(input_file,2,4,i)
        y.append(len(lsh_))
    
    plt.bar(x, y)
    plt.xlabel('row values')
    plt.ylabel('number of similar documents')
    plt.show()

lsh_count("sorgu.txt")